### Header

In [48]:
# import libraries

# maths
import numpy as np
import pandas as pd
#import scipy.stats as stats
#from pandas.api.types import is_numeric_dtype

# visual
#from matplotlib_venn import venn2
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

# modelling
# from sklearn import linear_model
# from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression
# from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
# from sklearn.preprocessing import StandardScaler,PolynomialFeatures
# from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
# from sklearn.pipeline import make_pipeline,Pipeline

# nlp
# from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from nltk.stem import PorterStemmer,WordNetLemmatizer
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# import spacy
# from spacy.tokens import Doc

# web
#import requests
#import json

# others
import os
import re
import time
import datetime as datetime

In [49]:
# file paths

input_path = '../data/2_input/'
clean_path = '../data/3_clean/'
output_path = '../data/4_output/'

image_path = '../images/'

### Import Data

In [50]:
train = pd.read_csv(input_path+'train.csv')
test = pd.read_csv(input_path+'test.csv')


### Inspect Data

In [51]:
train.head()


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [52]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


### Clean Data

In [53]:
def create_yr(x): 
    return x.split('-')[0] 

def create_mth(x): 
    return x.split('-')[1] 

def create_day(x): 
    return x.split('-')[2] 

def rename_columns (columns):
    return [column.lower() for column in columns]

def clean_data(df): 
    df['year'] = df.Date.apply(create_yr)
    df['month'] = df.Date.apply(create_mth)
    df['day'] = df.Date.apply(create_day)
    df['latitude'] = df.Latitude.apply(int)
    df['longitude'] = df.Longitude.apply(int)
    df.drop(['Address', 'AddressNumberAndStreet', 'AddressAccuracy', 'Date', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df.columns = rename_columns(df.columns)
    return df

train = clean_data(train)
test = clean_data(test)

### Output Data

In [54]:
train.head()

,species,block,street,trap,nummosquitos,wnvpresent,year,month,day,latitude,longitude
0,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,1,0,2007,05,29,41,-87
1,CULEX RESTUANS,41,N OAK PARK AVE,T002,1,0,2007,05,29,41,-87
2,CULEX RESTUANS,62,N MANDELL AVE,T007,1,0,2007,05,29,41,-87
3,CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,1,0,2007,05,29,41,-87
4,CULEX RESTUANS,79,W FOSTER AVE,T015,4,0,2007,05,29,41,-87


In [55]:
test.head()

,id,species,block,street,trap,year,month,day,latitude,longitude
0,1,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
1,2,CULEX RESTUANS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
2,3,CULEX PIPIENS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
3,4,CULEX SALINARIUS,41,N OAK PARK AVE,T002,2008,06,11,41,-87
4,5,CULEX TERRITANS,41,N OAK PARK AVE,T002,2008,06,11,41,-87


In [62]:
train[train.nummosquitos==50].shape

(1019, 11)

In [63]:
train[train.nummosquitos==50]

,species,block,street,trap,nummosquitos,wnvpresent,year,month,day,latitude,longitude
293,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,50,0,2007,07,11,41,-87
295,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,50,0,2007,07,11,41,-87
350,CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,50,0,2007,07,11,41,-87
351,CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,50,0,2007,07,11,41,-87
353,CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,50,0,2007,07,11,41,-87
529,CULEX PIPIENS/RESTUANS,12,S DOTY AVE,T115,50,0,2007,07,18,41,-87
530,CULEX PIPIENS/RESTUANS,10,S STONY ISLAND AVE,T138,50,0,2007,07,18,41,-87
531,CULEX PIPIENS/RESTUANS,10,S STONY ISLAND AVE,T138,50,0,2007,07,18,41,-87
533,CULEX RESTUANS,10,S STONY ISLAND AVE,T138,50,0,2007,07,18,41,-87
547,CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,50,0,2007,07,18,41,-87


In [59]:
train[train.duplicated()]

,species,block,street,trap,nummosquitos,wnvpresent,year,month,day,latitude,longitude
99,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,1,0,2007,06,26,41,-87
295,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,50,0,2007,07,11,41,-87
351,CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,50,0,2007,07,11,41,-87
353,CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,50,0,2007,07,11,41,-87
512,CULEX RESTUANS,33,E RANDOLPH ST,T044,1,0,2007,07,18,41,-87
531,CULEX PIPIENS/RESTUANS,10,S STONY ISLAND AVE,T138,50,0,2007,07,18,41,-87
548,CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,50,0,2007,07,18,41,-87
587,CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,50,0,2007,07,19,41,-87
588,CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,50,0,2007,07,19,41,-87
589,CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,50,0,2007,07,19,41,-87


In [64]:
col = [col for col in train.columns if col not in ['nummosquitos','wnvpresent']]

In [65]:
col

['species',
 'block',
 'street',
 'trap',
 'year',
 'month',
 'day',
 'latitude',
 'longitude']

In [70]:
train.groupby(train.columns.tolist(),as_index=False).size()

species          block  street               trap  nummosquitos  wnvpresent  year  month  day  latitude  longitude
CULEX ERRATICUS  10      W OHARE AIRPORT     T900  7             0           2013  09     06   41        -87          1
CULEX PIPIENS    10      E 67TH ST           T073  1             0           2007  08     01   41        -87          2
                                                                                   09     19   41        -87          1
                                                                                          24   41        -87          1
                                                                             2009  06     02   41        -87          1
                                                                                   07     17   41        -87          1
                                                                                   08     13   41        -87          1
                                             

In [66]:
train.duplicated(col).sum()

2031

In [67]:
train[train.duplicated(col)]

,species,block,street,trap,nummosquitos,wnvpresent,year,month,day,latitude,longitude
99,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,1,0,2007,06,26,41,-87
281,CULEX RESTUANS,79,W FOSTER AVE,T015,2,0,2007,07,11,41,-87
294,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,35,0,2007,07,11,41,-87
295,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,50,0,2007,07,11,41,-87
296,CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,8,0,2007,07,11,41,-87
298,CULEX RESTUANS,22,W 113TH ST,T086,8,0,2007,07,11,41,-87
316,CULEX RESTUANS,36,N PITTSBURGH AVE,T011,2,0,2007,07,11,41,-87
319,CULEX PIPIENS/RESTUANS,72,N OKETO AVE,T016,14,0,2007,07,11,42,-87
321,CULEX RESTUANS,72,N OKETO AVE,T016,4,0,2007,07,11,42,-87
351,CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,50,0,2007,07,11,41,-87


In [39]:
test.duplicated().sum()

0

In [35]:
train.to_csv(clean_path+'train_clean.csv')
test.to_csv(clean_path+'test_clean.csv')